In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

''''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       #print(os.path.join(dirname, filename))'''

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"'import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n       #print(os.path.join(dirname, filename))"

In [3]:
!ls /kaggle/input/intel-image-classification/seg_train/seg_train/

buildings  forest  glacier  mountain  sea  street


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout,Activation
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np


from tensorflow.keras.models import load_model
from keras.utils import np_utils
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import optimizers


Using TensorFlow backend.


In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,  # normalizing the image b/w 0 to 1
        shear_range=0.2,   # shear transformation
        zoom_range=0.2,     # zooming the image by 20%
        horizontal_flip=True)  # rotating
 
val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/intel-image-classification/seg_train/seg_train/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        '/kaggle/input/intel-image-classification/seg_test/seg_test/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
test_generator=test_datagen.flow_from_directory(
    '/kaggle/input/intel-image-classification/seg_pred/',
        target_size=(224, 224),
        batch_size=32,
        class_mode=None)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 7301 images belonging to 1 classes.


In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
import tensorflow.keras

inception=InceptionV3(include_top=False,weights='imagenet',input_shape=(224,224,3))
output1=inception.layers[-1].output
output1=tensorflow.keras.layers.Flatten()(output1)
inception=Model(inception.input,output1)
for layers in inception.layers:
  layers.trainable=False
inception.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_____________________________

In [7]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, InputLayer,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

model2 = Sequential()
model2.add(inception)
# model2.add(Conv2D(256,kernel_size=3,padding='same',activation='relu',input_shape=(224,224,3)))
# model2.add(MaxPool2D(pool_size=(2,2)))
# model2.add(Dropout(0.25))
# model2.add(Flatten())
model2.add(Dense(1024, activation='relu', input_dim=(224,224,3)))
# model2.add(Dense(1024,activation='relu'))
# model2.add(BatchNormalization())
model2.add(Dropout(0.25))
''''model2.add(Dense(256, activation='relu'))
# model2.add(BatchNormalization())
model2.add(Dense(64, activation='relu'))
# model2.add(Dropout(0.3))'''
model2.add(Dense(6, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 51200)             21802784  
_________________________________________________________________
dense (Dense)                (None, 1024)              52429824  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6150      
Total params: 74,238,758
Trainable params: 52,435,974
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history=model2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5
)

Epoch 1/5
438/438 [==============================] - 246s 561ms/step - loss: 2.1534 - accuracy: 0.8441 - val_loss: 0.2828 - val_accuracy: 0.8921
Epoch 2/5
438/438 [==============================] - 222s 507ms/step - loss: 0.3300 - accuracy: 0.8844 - val_loss: 0.3687 - val_accuracy: 0.8790
Epoch 3/5
438/438 [==============================] - 221s 504ms/step - loss: 0.3184 - accuracy: 0.8932 - val_loss: 0.2873 - val_accuracy: 0.8968
Epoch 4/5
438/438 [==============================] - 222s 506ms/step - loss: 0.2831 - accuracy: 0.9006 - val_loss: 0.2826 - val_accuracy: 0.8928
Epoch 5/5
438/438 [==============================] - 213s 485ms/step - loss: 0.2791 - accuracy: 0.9027 - val_loss: 0.2742 - val_accuracy: 0.9005


In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(20,10))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()
fig.savefig('../Accuracy_curve_CNN_256.jpg')

fig = plt.figure(figsize=(20,10))
plt.plot(loss)
plt.plot(val_loss)
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
fig.savefig('../loss_curve_CNN_256.jpg')


In [10]:
test_generator.reset()
pred=model2.predict_generator(test_generator,
verbose=1)

229/229 [==============================] - 34s 148ms/step


In [ ]:
print(pred)

In [16]:
import matplotlib.pyplot as plt
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [ print(k) for k in predicted_class_indices]



''''filenames=test_generator.filenames
results=pd.DataFrame({"Image":filenames,
                      "target":predictions})
results.to_csv("results.csv",index=False)
0.2..
106 '''

3
4
3
3
0
3
5
3
1
4
5
4
3
0
0
2
5
5
0
3
5
4
3
0
3
1
3
4
0
4
5
0
4
1
5
0
5
5
0
1
1
1
4
0
0
0
3
2
4
5
2
3
0
3
5
1
1
5
3
2
1
3
3
2
5
3
4
3
4
2
3
5
1
5
4
4
1
5
2
2
5
4
5
0
4
3
3
3
5
3
0
5
1
4
3
5
3
2
0
1
2
2
5
5
2
5
3
3
1
4
3
5
5
4
3
1
1
1
3
3
2
3
2
4
4
4
5
3
5
3
2
5
1
0
5
3
0
5
2
3
3
4
5
0
5
2
3
0
4
4
4
4
5
0
0
0
4
4
2
2
1
4
0
3
4
3
3
3
0
1
0
5
0
5
3
5
2
0
2
3
2
5
1
4
1
0
4
2
3
0
2
4
0
0
2
2
1
0
2
2
5
1
5
3
0
1
3
1
4
3
3
2
1
0
1
3
0
3
3
2
0
3
3
1
1
0
5
5
4
2
1
3
1
2
0
3
2
3
4
5
1
4
4
2
1
0
3
2
2
0
4
0
1
1
4
0
0
0
1
3
3
5
0
4
4
0
4
1
2
3
1
4
1
3
1
2
2
3
0
4
4
5
4
3
2
1
0
3
2
3
4
2
4
1
2
3
4
4
5
3
2
5
0
0
2
1
1
0
1
4
5
3
3
5
5
4
3
2
5
0
3
3
3
1
1
3
2
2
3
2
4
2
5
3
3
5
2
5
1
2
0
5
3
1
0
3
5
1
5
0
0
2
4
0
1
1
5
2
1
1
1
2
3
1
0
3
4
0
2
3
4
4
2
4
0
3
3
4
5
3
0
0
5
0
0
5
3
3
3
3
3
1
2
4
0
1
1
0
3
3
0
1
3
4
1
2
0
0
2
1
3
2
1
1
4
2
3
2
3
5
4
5
2
4
4
0
2
2
3
5
5
4
1
0
0
5
3
5
1
4
1
3
2
3
2
2
3
2
0
4
4
5
0
2
3
2
4
0
5
2
2
3
2
2
5
4
5
3
0
0
4
3
1
2
3
3
5
2
5
4
5
1
1
0
3
0
3
2
0
4
0
5
1
3
3
0
3
1
4
1


4
5
3
5
3
0
3
4
0
3
1
3
4
2
4
3
5
5
0
2
1
2
0
3
0
0
1
2
4
1
5
1
5
1
2
5
0
2
1
5
5
3
3
5
3
5
0
1
3
3
1
2
3
0
5
3
2
1
1
1
4
3
2
3
4
3
4
4
5
3
1
3
1
5
3
3
0
1
3
4
4
1
0
0
0
5
0
2
4
0
3
1
1
1
5
2
3
4
2
2
3
0
5
0
4
0
1
5
3
1
3
3
2
1
3
0
2
0
3
5
4
3
0
3
4
4
4
5
3
3
3
3
3
4
1
3
2
4
1
4
1
1
3
4
3
0
4
2
4
0
5
2
4
0
4
0
1
5
3
3
3
4
1
2
4
1
4
3
2
1
2
3
1
3
5
2
2
2
5
5
1
1
2
5
2
5
0
0
1
5
2
5
5
3
5
0
0
0
1
0
4
4
1
5
1
3
5
4
4
5
1
1
3
5
5
1
0
0
3
3
2
4
3
4
3
5
0
4
5
3
3
5
5
2
0
3
1
4
3
3
3
4
5
4
1
1
5
1
4
1
2
1
2
0
2
5
1
5
0
1
2
2
1
0
1
4
3
0
4
4
5
1
2
4
0
4
0
5
3
3
4
3
5
1
4
3
1
5
4
0
0
2
0
4
2
1
2
2
1
1
3
1
4
4
3
2
0
4
1
3
2
5
5
0
0
0
1
0
4
3
0
1
1
1
1
0
0
3
4
4
1
0
1
4
5
4
5
5
2
0
2
4
3
3
4
0
0
3
4
4
1
4
3
2
2
1
4
0
2
5
5
2
0
1
4
3
1
0
1
3
3
5
4
4
4
5
1
5
1
5
2
5
2
5
1
0
3
4
4
1
5
0
5
5
4
1
3
0
5
1
0
3
0
0
1
3
5
2
0
1
3
1
4
3
3
0
4
4
4
0
0
0
1
5
0
5
1
2
3
5
3
3
2
2
0
3
0
4
2
4
4
0
4
3
1
2
5
5
4
5
2
3
2
4
5
1
4
4
4
1
2
4
1
3
5
1
3
5
2
4
3
4
2
2
3
1
3
5
1
0
4
0
4
3
5
5
1
5
0
0
4
1
1
4
3
3
5
4
2
0


'\'filenames=test_generator.filenames\nresults=pd.DataFrame({"Image":filenames,\n                      "target":predictions})\nresults.to_csv("results.csv",index=False)\n0.2..\n106 '

In [12]:
for filenames in os.walk('/kaggle/input/intel-image-classification/seg_pred/seg_pred/'):
    print(filename)


NameError: name 'os' is not defined